# Week 11: Hierarchical clustering

## Goals 
- Examples of hierarchical clustering
- Perform hierarchical clustering
- Compare to $k$-means

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering

## Wine

Let's take a look at SLINK and CLINK using a wine data set.

Data set was obtained from [kaggle.com](https://www.kaggle.com/datasets/harrywang/wine-dataset-for-clustering/). The original [data set](https://archive.ics.uci.edu/dataset/109/wine) is publicly available with the [CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/legalcode) license. 

In [ ]:
df = pd.read_csv("data/wine-clustering.csv")
df.head()

We have 13 columns, so our data points live in $\mathbb{R}^{13}$.

Let's quickly run a PCA and project the data onto the first two principal components. 

Here is code to scale the data, but we could probably avoid it. We definitely need to translate the data, so that the mean is at the origin.

In [ ]:
def mat_to_rescaled_mat(Z):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(Z)
    return scaler.transform(Z)

We'll work with two data sets: `X` and `Z`.

- `X` : raw data 
- `Z` : scaled data (via StandardScaler in `sklearn`)

From `scikit-learn`'s [website](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html):

> Standardization of a dataset is a common requirement for many 
> machine learning estimators: they might behave badly if the 
> individual features do not more or less look like standard 
> normally distributed data (e.g. Gaussian with 0 mean and 
> unit variance).

In [ ]:
X = df.to_numpy()
Z = mat_to_rescaled_mat(X)
print("First 3 rows of X:")
print(X[:3,:])
print("\nFirst 3 rows of Z:")
print(Z[:3,:])

In [ ]:
pca = PCA(n_components=2)
pca.fit(Z)
Y = pca.transform(Z)
plt.figure(1)
_ = plt.scatter(Y[:,0], Y[:,1])

Let's also briefly look at a $k$-means clustering. 

First let's see if we can get any information from an elbow plot.

In [ ]:
def MyKMeans(k, data):
    return KMeans(n_clusters=k, n_init="auto").fit(data)

def ElbowMethodPlot(data, N):
    import matplotlib.pyplot as plt

    def WCSS(data, kmeans):
        from functools import reduce
        ord_cents = map(lambda i: kmeans.cluster_centers_[i], kmeans.labels_)
        tups = zip(data, ord_cents)
        dist2 = lambda t: np.linalg.norm(t[0] - t[1])**2
        return reduce(lambda x, y: x + dist2(y), tups, 0)

    KMEANS = [MyKMeans(k, data) for k in range(1, N + 1)]
    WCSS_vals = [WCSS(data, kmeans) for kmeans in KMEANS]

    plt.figure(1)
    plt.grid()
    plt.plot(WCSS_vals)
    plt.xticks(ticks=range(N), labels=range(1, N+1))
    return plt

In [ ]:
plot = ElbowMethodPlot(Z, 15)

It's not a strong indicator, but $k=3$ seems like a reasonable choice.

We'll run a $3$-means algorithm on the original data, and then color the data points in $\mathbb{R}^2$ after PCA.

In [ ]:
kmeans_data = MyKMeans(3, Z)
plt.clf()
_ = plt.scatter(Y[:,0], Y[:,1], c=kmeans_data.labels_)

Now we will do this with a hierarchical clustering algorithm.

We will use `scikit-learn` to do hierarchical clustering, but `scipy` has a very good function as well. Moreover, `scipy` has a `dendrogram` function, which is built around their own clustering algorithm.

Instead, we will just use the following dendrogram plotting function.

In [ ]:
def plot_dendrogram(model, **kwargs):
    import numpy as np
    # from matplotlib import pyplot as plt
    from scipy.cluster.hierarchy import dendrogram
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [ ]:
def SLINK(data):
    return AgglomerativeClustering(n_clusters=None, distance_threshold=0, linkage="single").fit(data)

def CLINK(data):
    return AgglomerativeClustering(n_clusters=None, distance_threshold=0, linkage="complete").fit(data)

def ALINK(data):
    return AgglomerativeClustering(n_clusters=None, distance_threshold=0, linkage="average").fit(data)

def WLINK(data):    # Formula for WARD is given later
    return AgglomerativeClustering(n_clusters=None, distance_threshold=0, linkage="ward").fit(data)

def HCluster(k, data, link="ward"):
    return AgglomerativeClustering(n_clusters=k, linkage=link).fit(data)

In [ ]:
plot_dendrogram(WLINK(X), truncate_mode="level", p=4)	
# p is where we cut off our dendrogram

Let's see how hierarchical clustering compares with out $k$-means clusters.

In [ ]:
hclus_data = HCluster(3, Z, link="ward")
plt.clf()
_ = plt.scatter(Y[:,0], Y[:,1], c=hclus_data.labels_)

It was at this stage that I dug through the original data for more information about the wine.

I found the following quote:

> These data are the results of a chemical analysis of wines grown in the same region in Italy but derived from three different cultivars.
> 
> The analysis determined the quantities of 13 constituents found in each of the three types of wines. 

🤯🤯🤯

## Ward-linkage

Recall we have given formulae for the "single", "complete" and "average" linkages.

If $C_1$ and $C_2$ are finite (disjoint) subsets of $\mathbb{R}^m$, then 
1. `SINGLE` is given as
$$
    d(C_1, C_2) = \min\left\{ \| x - y\| : x\in C_1, y\in C_2 \right\},
$$
2. `COMPLETE` is given as 
$$
    d(C_1, C_2) = \max\left\{ \| x - y\| : x\in C_1, y\in C_2 \right\},
$$
3. `AVERAGE` is given as 
$$
    d(C_1, C_2) = \sum_{x \in C_1} \sum_{y\in C_2} \dfrac{\|x - y\|}{\# C_1 \cdot \# C_2},
$$
4. `WARD` is given as 
$$
    d(C_1, C_2) = \sum_{x\in C_1}\sum_{y\in C_2} \dfrac{\|x - y\|^2}{\# C_1 + \# C_2} - \sum_{x\in C_1}\sum_{y\in C_1} \dfrac{\|x - y\|^2}{\# C_1} - \sum_{x\in C_2}\sum_{y\in C_2} \dfrac{\|x - y\|^2}{\# C_2}.
$$

The idea behind the `WARD` linkage is to join clusters $C_1$ and $C_2$ only if they are the pair that minimizes the WCSS (Within-Cluster Sum of Squares). 

This is the same measurement we used to generate Elbow plots.

We will use `WARD` here in this notebook, but you do not need to commit it to memory. 

## Examples with hierarchical clustering

Let's go through the same data sets we went through with $k$-means and see how hierarchical compares.

We will use `HCluster` defined above, and for plotting let's use the following code. Similar format as `Voronoi` from Week10.

In [ ]:
def plot_hcluster(data, hcluster=None):
    import matplotlib.cm as cm
    import matplotlib.pyplot as plt 
    plt.figure(1)
    plt.clf()
    if hcluster:
        plt.scatter(data[:,0], data[:,1], c=hcluster.labels_)
    else:
        plt.scatter(data[:,0], data[:,1], c="black")
    return plt 

### Three clear clusters

First up is `data/three_clusters_clear.csv`. Just to remind us what it looks like.

In [ ]:
data_3c = pd.read_csv("data/three_clusters_clear.csv").to_numpy()
plot = plot_hcluster(data_3c)

In [ ]:
hcluster_3c = HCluster(3, data_3c, link="single")
plot = plot_hcluster(data_3c, hcluster=hcluster_3c)

What does the corresponding dendrogram look like?

In [ ]:
plot = plot_dendrogram(SLINK(data_3c), truncate_mode="level", p=3)

We can see the three clusters (for `p=3`):
- there are 58 points in orange: 54 points plus points labeled 0, 150, 81, and 114.
- there are two sets of 58 points in green.
  - left: 55 points plus 2 points plus a point labeled 104.
  - right: 56 points plus points labeled 166 and 16.

### Three vague clusters

In [ ]:
data_3v = pd.read_csv("data/three_clusters_vague.csv").to_numpy()
plot = plot_hcluster(data_3v)

In [ ]:
hcluster_3v = HCluster(3, data_3v, link="single")
plot = plot_hcluster(data_3v, hcluster=hcluster_3v)

### Two unbalanced clusters

It was only now that I realised I had a typo in my `csv` file...

In [ ]:
data_2u = pd.read_csv("data/two_clusters_unbalanced.csv").to_numpy()
plot = plot_hcluster(data_2u)

In [ ]:
hcluster_2u = HCluster(2, data_2u, link="single")
plot = plot_hcluster(data_2u, hcluster=hcluster_2u)

### The 'impossible' data set

We can see hierarchical clustering shine compared to $k$-means with this 'impossible' data set.

In [ ]:
data_i = pd.read_csv("data/impossible.csv").to_numpy()
plot = plot_hcluster(data_i)

The number of clusters is not so clear-cut as the previous examples. Let's try a few.

In [ ]:
hcluster_i = HCluster(2, data_i, link="single")
plot = plot_hcluster(data_i, hcluster=hcluster_i)

## Try it yourself!

There are five data sets to play around with. Feel free to use the functions here: 
- `plot_dendrogram` with:
  - `SLICK`, `CLINK`, `ALINK`, `WLINK`,
- `HCluster`, 
- `plot_hcluster`.

Here are some question to consider.
1. How many clusters are there in the data? 
2. Do the clusters seem to be unambiguous? 
3. What other intrepretations are there?
4. Does the hierarchical clustering do what you expect? Why? 
5. Does one linkage perform far better (or far worse) than the others? Why do you think?

Discuss this with a neighbour.

Data sets:
- `extra_1.csv`
- `extra_2.csv`
- `extra_3.csv`
- `extra_4.csv`
- `extra_5.csv`